# Projeto de ciência de dados 2023/2024

## Este bloco de notas apresenta a realização das 3 peimeiras fazes do ciclo de anlize de dados para o projeto de ciência de dados de 2023/2024.

### Formulação do Problema

#### Num contexto global focado nos Objetivos de Desenvolvimento Sustentável da ONU, destaca-se a meta primordial de reduzir substancialmente a mortalidade infantil até 2030. O objetivo é claro: eliminar as mortes evitáveis de recém-nascidos e crianças menores de 5 anos, visando uma taxa de mortalidade infantil abaixo de 25 por cada 1.000 nascidos vivos em todos os países.

#### Num cenário onde a busca pela concretização dos Objetivos de Desenvolvimento Sustentável da ONU é imperativa, destaca-se a necessidade urgente de reduzir a mortalidade infantil e materna. Em 2017, a mortalidade materna, responsável por 295.000 óbitos durante e após a gravidez, permanece uma preocupação alarmante, sobretudo nas regiões com recursos limitados, onde ocorrem 94 por cento dessas perdas. Esses dados sublinham a necessidade crítica de estratégias inovadoras e acessíveis para evitar essas tragédias evitáveis.

Neste contexto, os Cardiotocogramas (CTGs) emergem como ferramentas essenciais na avaliação da saúde fetal, proporcionando uma abordagem simples e economicamente viável. Ao enviar pulsos de ultrassom e analisar as respostas, os CTGs oferecem informações cruciais sobre a frequência cardíaca fetal (FCF), movimentos fetais, contrações uterinas, entre outros aspetos, permitindo intervenções proativas para mitigar riscos à saúde materna e infantil.

Este relatório aborda de maneira abrangente o papel crucial dos CTGs na promoção da saúde materno-infantil. A análise dos dados coletados por esses dispositivos busca proporcionar uma compreensão aprofundada das ações empreendidas, justificando cada passo do processo. Desta forma, almejamos contribuir significativamente para o conhecimento e a prática na prevenção da mortalidade infantil e materna, alinhando-se aos objetivos globais de desenvolvimento sustentável estabelecidos pela comunidade internacional.